In [1]:
import pandas as pd
import numpy as np
import re
import hanja

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./src/네이버금융종목뉴스기사본문_네이버_전체_관련뉴스제거.csv')
df.head()

,날짜,제목,정보제공,링크,기사
0,2022-05-10,"네이버, AI 프로젝트 챌린지 참가자 모집한다",파이낸셜뉴스,https://finance.naver.com/item/news_read.naver...,\n‘CLOVA AI RUSH 2022’ 서비스 기획도 신설네이버 데이터와 인프라 ...
1,2022-05-10,"레드아이스·부천시, 네이버웹툰과 총상금 1억 5000만 원 웹툰 공모전...",이데일리,https://finance.naver.com/item/news_read.naver...,\n\t\t\t [이데일리 이윤정 기자] 웹툰 전문 제작사 레드아이스 스튜디오와 부...
2,2022-05-09,성장주 카카오·네이버의 추락…주가 반토막에 증권가 전망도 ‘흐림’,조선비즈,https://finance.naver.com/item/news_read.naver...,"\n한달 새 카카오 -19.7%, 네이버 -17.6% 급락1분기 부진한 실적이 부정..."
3,2022-05-09,"[단독] 네이버, 웹3 스타트업 등에 1000억 베팅",서울경제,https://finance.naver.com/item/news_read.naver...,\n4년 만에 티비티 3호 벤처펀드에 추가 출자 검토네이버의 벤처펀드 출자 사례 중...
4,2022-05-09,"하림, 10일 네이버 쇼핑라이브 특가 판매",조세일보,https://finance.naver.com/item/news_read.naver...,\n 종합식품기업 하림이 10일 오전 11시부터 1시간 동안 네이버 쇼핑라이...


In [3]:
df['기사'] = df['기사'].str.replace('\n|\t|\xa0|\u200b',' ') # \n,\t,\xa0,\u200b 삭제
df['기사'] = df['기사'].str.replace("'|‘|’",'"') # 따옴표 모두 큰따옴표로 통일
df['기사'] = df['기사'].str.replace('↑','상승').str.replace('↓','하락') # ↑↓ 치환
df['기사'] = df['기사'].str.replace('\([^\(\)]*?\)|\[[^\[\]]*?\]|<[^<>]*?>',' ').str.replace('\([^\(\)]*?\)|\[[^\[\]]*?\]|<[^<>]*?>',' ') # (),[],<> 제거
df['기사'] = df['기사'].str.replace('·+',',') # ·을 ,로 치환
df['기사'] = df['기사'].str.replace('[가-힣]{2,3} ?기자',' ') # 홍길동 기자 삭제
df['기사'] = df['기사'].str.replace('[a-z0-9]+@',' ') # 기자 태그 삭제
df['기사'] = df['기사'].str.replace('사진=|사진 ?제공=',' ') # 사진 태그 삭제

art_list = [] # 공통 광고 제거 및 한자번역
for i in range(len(df['기사'].to_list())):
    art = df['기사'].to_list()[i]
    info = df['정보제공'].to_list()[i]
    rm = f'{info} 관련뉴스해당 언론사에서.*'
    art = re.sub(rm,' ',art) # 공통 광고 제거
    art = hanja.translate(art, 'substitution') # 한자 한글로 번역
    art_list.append(art)
df['기사'] = art_list

df['기사'] = df['기사'].str.replace('[^ 0-9ㄱ-ㅎ가-힣a-zA-Z"%&,.~-]',' ') # 특수문자 삭제
df['기사'] = df['기사'].str.replace('\.+','.') # . 2개 이상이면 1개만 남김
df['기사'] = df['기사'].str.replace(' +',' ') # 공백 2개 이상이면 1개만 남김
df['기사'] = df['기사'].str.strip() # 좌우 공백 제거
df.head()

,날짜,제목,정보제공,링크,기사
0,2022-05-10,"네이버, AI 프로젝트 챌린지 참가자 모집한다",파이낸셜뉴스,https://finance.naver.com/item/news_read.naver...,"""CLOVA AI RUSH 2022"" 서비스 기획도 신설네이버 데이터와 인프라 활용..."
1,2022-05-10,"레드아이스·부천시, 네이버웹툰과 총상금 1억 5000만 원 웹툰 공모전...",이데일리,https://finance.naver.com/item/news_read.naver...,"웹툰 전문 제작사 레드아이스 스튜디오와 부천시가 네이버웹툰과 함께하는 ""2022 모..."
2,2022-05-09,성장주 카카오·네이버의 추락…주가 반토막에 증권가 전망도 ‘흐림’,조선비즈,https://finance.naver.com/item/news_read.naver...,"한달 새 카카오 -19.7%, 네이버 -17.6% 급락1분기 부진한 실적이 부정적 ..."
3,2022-05-09,"[단독] 네이버, 웹3 스타트업 등에 1000억 베팅",서울경제,https://finance.naver.com/item/news_read.naver...,4년 만에 티비티 3호 벤처펀드에 추가 출자 검토네이버의 벤처펀드 출자 사례 중 역...
4,2022-05-09,"하림, 10일 네이버 쇼핑라이브 특가 판매",조세일보,https://finance.naver.com/item/news_read.naver...,종합식품기업 하림이 10일 오전 11시부터 1시간 동안 네이버 쇼핑라이브 채널에서 ...


In [4]:
df.to_csv("./src/네이버금융종목뉴스기사본문_네이버_전체_관련뉴스제거_전처리후.csv", index=False)